In [1]:
""" Playing with Examples given in
    the Python Data Science Handbook
    Chapter 5, Section 3:
    Hyperparameters and Model Validation
    https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html
"""

from sklearn.datasets import load_iris
iris = load_iris()
X = iris.data
y = iris.target


In [14]:
""" Let's look at this first Iris dataset a little
"""
print('Size of Dataset:')
print(X.shape)
print(y.shape)

print('\nExample data:')
print(X[48:52])
print(y[48:52])

Size of Dataset:
(150, 4)
(150,)

Example data:
[[ 5.3  3.7  1.5  0.2]
 [ 5.   3.3  1.4  0.2]
 [ 7.   3.2  4.7  1.4]
 [ 6.4  3.2  4.5  1.5]]
[0 0 1 1]


In [18]:
""" Model Validation, the Ridiculously Wrong Way
    (training and testing on the same data)
    
    Also, the "k-nearest neighbors classifier"
    algorithm relates each new point to the
    closest point in the original set.
"""
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=1)
model.fit(X, y)
y_model = model.predict(X)

from sklearn.metrics import accuracy_score

accuracy_score(y, y_model)

1.0